## Google OCR API

### Step1. 구글의 파이썬 API 인터페이스 모듈을 설치  

`$ pip install --upgrade google-api-python-client`  

`$ pip install google-cloud-vision`

### Step2. Google Cloud Vision API 사용
아래 링크의 설명을 참고하여 서비스 계정 및 인증키를 생성합니다. 브라우저에서 다운로드한 인증키는 다음 경로에 my_google_api_key.json이라는 파일명으로 저장해 둡시다. (파일은 처음에 sheet-contents-로 시작되는 이름으로 자동 저장됩니다.)   

[구글 클라우드 비전 API 사용하기](http://egloos.zum.com/mcchae/v/11342622)

`$ cp ~/Downloads/sheet-contents-xxxx.json ~/aiffel/ocr_python/my_google_api_key.json`

### Step3. 인증키 경로 등록 후 커널 재기동  

터미널을 열고 아래와 같이 인증키 경로 변수를 등록한 후 커널을 종료하고 재기동합니다.  
`$ export GOOGLE_APPLICATION_CREDENTIALS=$HOME/aiffel/ocr_python/my_google_api_key.json`  

만약 구글 API를 계속 사용하고 싶다면 아래와 같이 환경변수에 등록해 주면 위와 같이 매번 경로 변수 설정을 하지 않아도 됩니다.  
  
`$ echo "export GOOGLE_APPLICATION_CREDENTIALS=$HOME/aiffel/ocr_python/my_google_api_key.json" >> ~/.bashrc`

### Step4. API 사용 테스트
API를 활용하는 코드는 아래와 같습니다. 화면에서 테스트할 때 사용했던 이미지를 다시한번 사용해 보세요.

In [1]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
       print('\n"{}"'.format(text.description))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                 for vertex in text.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [2]:
# 다운받은 인증키 경로가 정확하게 지정되어 있어야 합니다. 
!ls -l $GOOGLE_APPLICATION_CREDENTIALS

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] =  os.getenv('HOME')+'/aiffel/ocr_python/my_google_api_key.json'

# 입력 이미지 경로를 지정해 주세요.
# (예시) path = os.getenv('HOME')+'/aiffel/ocr_python/test_image.png'
path = os.getenv("HOME") +'/1.AIFFEL_Study/Exploration/E18_OCR/pics/ocr1.png'

# 위에서 정의한 OCR API 이용 함수를 호출해 봅시다.
detect_text(path)

-rw-rw-r-- 1 aiffel-dj40 aiffel-dj40 2355  3월 16 14:10 /home/aiffel-dj40/aiffel/ocr_python/my_google_api_key.json
Texts:

"012345678
SABCDEFGH
IJKLMNOPQ
RSTUVWXYZ
012345678
9ABCDEFGH
IJKLMNOPO
RSTUVWXYZ
012345678
9ABCDEFGH
IJKLMNOP@
R
"

"012345678"

"SABCDEFGH"

"IJKLMNOPQ"

"RSTUVWXYZ"

"012345678"

"9ABCDEFGH"

"IJKLMNOPO"

"RSTUVWXYZ"

"012345678"

"9ABCDEFGH"

"IJKLMNOP@"

"R"
bounds: (5,555),(44,555),(44,594),(5,594)


이전에 구글 API 계정을 생성한 후 결제정보가 만료된 경우에는 위의 OCR API 호출시 billing 관련 오류가 날 수 있습니다. 오류메시지 아래쪽의 링크를 따라가서 결제계정을 재활성화해야 API 호출이 가능합니다.
혹은 다른 계정을 생성하고 인증키를 다시 받는 방법도 있습니다. 

## keras-ocr

keras-ocr은 텐서플로우의 케라스 API를 기반으로 이미지 속 문자를 읽는 End-to-End OCR을 할 수 있게 해줍니다. 공식 문서에도 나와 있듯, 검출 모델로는 네이버 데뷰 2018 영상에서 소개한 CRAFT(Character Region Awareness for Text Detection)를 사용하고, 인식 모델로는 앞에서 설명한 CRNN을 사용합니다.

주의 keras-ocr 은 tensorflow 버전 2.2.0 에서 구동됩니다. 2.3.X 이상 버전에서는 미리 학습된 모델에서 오류가 발생할 수 있으니 주의하세요. 

`$ pip list | grep tensorflow`    

만약 tensorflow 버전이 맞지 않다면 재설치를 해줍시다.  
`$ pip uninstall tensorflow`  

`$ pip install tensorflow==2.2.0`

이제 `keras-ocr`사용을 위해서 설치  
`$ pip install keras-ocr`

필요한 라이브러리인 `keras_ocr` 과 인식결과의 시각화를 위한 `matplotlib.pyplot` 를 불러옵니다. `keras_ocr.pipeline.Pipeline()` 는 인식을 위한 파이프라인을 생성하는데 이때 초기화 과정에서 미리 학습된 모델의 가중치(weight)를 불러오게 됩니다. 검출기와 인식기를 위한 가중치 하나씩을 불러오겠지요.  

[keras-ocr 공식문서](https://keras-ocr.readthedocs.io/en/latest/index.html)

❗️ 아래 과정은 GPU를 사용할 수 있습니다. 평소 GPU 사용 시 cuDNN 관련 에러가 있었다면 아래 명령어를 사용해 보세요. export TF_FORCE_GPU_ALLOW_GROWTH=true

(편집자 주) AIFFEL 학습환경 설치 과정에서 위 옵션이 아래와 같이 환경설정에 반영되어 있습니다.  
`$ echo "export TF_FORCE_GPU_ALLOW_GROWTH=true" >> ~/.bashrc`

확인을 위해서 터미널을 열어 다음과 같이 확인해 보시기 바랍니다. `true`가 출력되어야 환경설정에 반영되어 있는 것입니다. 만약 이 환경설정이 반영되어 있지 않으면 이후 코드 구동 과정에서 OOM(Out Of Memory) 에러가 날 수 있습니다.  

`$ echo $TF_FORCE_GPU_ALLOW_GROWTH`

In [ ]:
import matplotlib.pyplot as plt
import keras_ocr

# keras-ocr이 detector과 recognizer를 위한 모델을 자동으로 다운로드받게 됩니다. 
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /home/aiffel-dj40/.keras-ocr/craft_mlt_25k.h5


만들어둔 파이프라인의 `recognize()` 에 이미지를 몇 개 넣어줍니다.
이미지소스의 url을 사용할건데요. 이미지는 https://unsplash.com/s/photos/text 에서 가져왔습니다.

In [5]:
# 테스트에 사용할 이미지 url을 모아 봅니다. 추가로 더 모아볼 수도 있습니다. 
image_urls = [
  'https://source.unsplash.com/M7mu6jXlcns/640x460',
  'https://source.unsplash.com/6jsp4iHc8hI/640x460',
  'https://source.unsplash.com/98uYQ-KupiE',
  'https://source.unsplash.com/j9JoYpaJH3A',
  'https://source.unsplash.com/eBkEJ9cH5b4'
]

images = [ keras_ocr.tools.read(url) for url in image_urls]
prediction_groups = [pipeline.recognize([url]) for url in image_urls]

ResourceExhaustedError:  OOM when allocating tensor with shape[1,64,920,1280] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/basenet.slice1.0/Conv2D (defined at /home/aiffel-dj40/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/keras_ocr/detection.py:682) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_predict_function_6881]

Function call stack:
predict_function


이제 인식된 결과를 pyplot으로 시각화를 해봅니다.

사용이 매우 간단합니다! 내부적으로 `recognize()` 는 검출기와 인식기를 두고, 검출기로 바운딩 박스(bounding box, 문자가 있는 영역을 표시한 정보)를 검출한 뒤, 인식기가 각 박스로부터 문자를 인식하는 과정을 거치도록 합니다.  

[kera-ocr 파이프라인](https://github.com/faustomorales/keras-ocr/blob/master/keras_ocr/pipeline.py)

In [ ]:
# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for idx, ax in enumerate(axs):
    keras_ocr.tools.drawAnnotations(image=images[idx], 
                                    predictions=prediction_groups[idx][0], ax=ax)

(주의사항)
keras-ocr은 한글 데이터셋으로 훈련이 되어있지 않은 모델입니다. 한글 텍스트의 detection은 정상적으로 진행되더라도 recognition 결과가 엉뚱하게 나올 수 있음에 주의해 주세요.  
[https://github.com/faustomorales/keras-ocr/issues/101](https://github.com/faustomorales/keras-ocr/issues/101)

## 테서렉트

이번에는 테서랙트(Tesseract) 라이브러리로 이미지에서 문자를 인식해 보겠습니다. 테서랙트는 구글에서 후원하는 OCR 오픈소스 라이브러리로 현재는 버전 4와 Tessearct.js등으로 확장되는 등 많은 곳에서 사용되고 있습니다. 버전 4에서는 LSTM이 엔진에 추가되었고 현재 한국어를 포함한 116 개 국어를 지원하고 있습니다.

오픈소스라는 점은 여러분들이 원하는 프로젝트에 활용하기 쉽다는 것을 뜻하니, 직접 해 보면서 익혀두면 나중에 간단한 OCR 모델이 필요할 때 빠르게 활용할 수 있을 겁니다.

### Step1. 테서렉트 설치
우선 우분투에서 실행할 경우 터미널에서 아래 코드를 사용해 테서랙트 관련 패키지들을 설치해 주세요. 혹시 다른 운영체제를 사용하실 경우에는 아래 Tesseract Install Guide를 참고해주세요.  
[테서렉트 설치 가이드](https://github.com/tesseract-ocr/tesseract/wiki)  
   
$ sudo apt install tesseract-ocr  

$ sudo apt install libtesseract-dev

### Step2. 테서렉트 파이썬 wrapper 설치  
`Pytesseract`는 OS에 설치된 테서랙트를 파이썬에서 쉽게 사용할 수있도록 해주는 래퍼 라이브러리(wrapper library)입니다. 파이썬 내에서 컴퓨터에 설치된 테서랙트 엔진의 기능을 바로 쓸 수 있도록 해줍니다.

- 참고: [tesseract](https://pypi.org/project/pytesseract/)  
- 참고: [위키백과: 래퍼 라이브러리](https://ko.wikipedia.org/wiki/%EB%9E%98%ED%8D%BC_%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC)  

$ pip install pytesseract  

### 테서렉트로 문자 검출하고 이미지 자르기(detection)

In [ ]:
import os
import pytesseract
from PIL import Image
from pytesseract import Output
import matplotlib.pyplot as plt

# OCR Engine modes(–oem):
# 0 - Legacy engine only.
# 1 - Neural nets LSTM engine only.
# 2 - Legacy + LSTM engines.
# 3 - Default, based on what is available.

# Page segmentation modes(–psm):
# 0 - Orientation and script detection (OSD) only.
# 1 - Automatic page segmentation with OSD.
# 2 - Automatic page segmentation, but no OSD, or OCR.
# 3 - Fully automatic page segmentation, but no OSD. (Default)
# 4 - Assume a single column of text of variable sizes.
# 5 - Assume a single uniform block of vertically aligned text.
# 6 - Assume a single uniform block of text.
# 7 - Treat the image as a single text line.
# 8 - Treat the image as a single word.
# 9 - Treat the image as a single word in a circle.
# 10 - Treat the image as a single character.
# 11 - Sparse text. Find as much text as possible in no particular order.
# 12 - Sparse text with OSD.
# 13 - Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

def crop_word_regions(image_path='./images/sample.png', output_path='./output'):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    custom_oem_psm_config = r'--oem 3 --psm 3'
    image = Image.open(image_path)

    recognized_data = pytesseract.image_to_data(
        image, lang='kor',    # 한국어라면 lang='kor'
        config=custom_oem_psm_config,
        output_type=Output.DICT
    )
    
    top_level = max(recognized_data['level'])
    index = 0
    cropped_image_path_list = []
    for i in range(len(recognized_data['level'])):
        level = recognized_data['level'][i]
    
        if level == top_level:
            left = recognized_data['left'][i]
            top = recognized_data['top'][i]
            width = recognized_data['width'][i]
            height = recognized_data['height'][i]
            
            output_img_path = os.path.join(output_path, f"{str(index).zfill(4)}.png")
            print(output_img_path)
            cropped_image = image.crop((
                left,
                top,
                left+width,
                top+height
            ))
            cropped_image.save(output_img_path)
            cropped_image_path_list.append(output_img_path)
            index += 1
    return cropped_image_path_list


work_dir = os.getenv('HOME')+'/aiffel/ocr_python'
img_file_path = work_dir + '/test_image.png'   #테스트용 이미지 경로입니다. 본인이 선택한 파일명으로 바꿔주세요. 

cropped_image_path_list = crop_word_regions(img_file_path, work_dir)

위에서 구현한 crop_word_regions() 함수는 여러분이 선택한 테스트 이미지를 받아서, 문자 검출을 진행한 후, 검출된 문자 영역을 crop한 이미지로 만들어 그 파일들의 list를 리턴하는 함수입니다.

기본적으로 pytesseract.image_to_data() 를 사용합니다. 파이썬에서 편하게 사용하기 위해서 pytesseract 의 Output 을 사용해서 결과값의 형식을 딕셔너리(DICT) 형식으로 설정해주게 됩니다. 이렇게 인식된 결과는 바운딩 박스의 left, top, width, height 정보를 가지게 됩니다. 바운딩 박스를 사용해 이미지의 문자 영역들을 파이썬 PIL(pillow) 또는 opencv 라이브러리를 사용해 잘라(crop)서 cropped_image_path_list에 담아 리턴하였습니다.

(주의) 위 코드에서 lang='kor' 로 바꾸면 에러가 발생합니다. 테서랙트의 언어팩을 설치해야 정상동작하게 됩니다. 

$ sudo apt install tesseract-ocr-kor

### 테서렉트로 잘린 이미지에서 단어 인식하기

이제 문자 인식을 해 볼 차례입니다. 검출된 바운딩 박스 별로 잘린 이미지를 넣어주면 영역별 텍스트가 결과값으로 나오는 image_to_string()를 사용하게 됩니다.

이렇게 인식된 결과가 실제 이미지와 맞는지 확인해 봅시다.

In [ ]:
def recognize_images(cropped_image_path_list):
    custom_oem_psm_config = r'--oem 3 --psm 7'
    
    for image_path in cropped_image_path_list:
        image = Image.open(image_path)
        recognized_data = pytesseract.image_to_string(
            image, lang='eng',    # 한국어라면 lang='kor'
            config=custom_oem_psm_config,
            output_type=Output.DICT
        )
        print(recognized_data['text'])
    print("Done")

# 위에서 준비한 문자 영역 파일들을 인식하여 얻어진 텍스트를 출력합니다.
recognize_images(cropped_image_path_list)

블러처리  
https://whereisend.tistory.com/143